Coverages Hacks
===============

In this lab, we'll be looking at Cubewerx test server, and especially how it handles OGC API Coverage.

The work is in python. We'll need a couple of python tools:

In [1]:
import json
import requests

Landing Page
------------

Fetch the landing page using a HTTP GET request and load it up as (nested) python dictionary.

In [2]:
landingPageResponse = requests.get('https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa')
landingPage = json.loads(landingPageResponse.text)

Lets look at what that looks like, turning it back in to pretty-printed JSON.

In [3]:
print(json.dumps(landingPage, indent=4))

{
    "title": "Daraa",
    "description": "This data store is offered by CubeWerx Inc. as a demonstration of its in-progress OGC API implementation.",
    "links": [
        {
            "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa?f=json",
            "rel": "self",
            "type": "application/json",
            "title": "this OGC API landing page as JSON"
        },
        {
            "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa?f=xml",
            "rel": "alternate",
            "type": "application/xml",
            "title": "this OGC API landing page as XML"
        },
        {
            "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa?f=html",
            "rel": "alternate",
            "type": "text/html",
            "title": "this OGC API landing page as HTML"
        },
        {
            "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa/api?f=json",
            "rel

Conformance
-----------
Lets look at the conformance statement for this server. First, lets get the appropriate link from the landing page result.


In [4]:

conformanceLink = next(link for link in landingPage['links'] if (link['rel'] == 'conformance' and link['type'] == 'application/json'))
print(json.dumps(conformanceLink, indent=4))


{
    "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa/conformance?f=json",
    "rel": "conformance",
    "type": "application/json",
    "title": "the OGC API conformance classes supported"
}


Fetch the conformance page and load it up.


In [5]:
conformancePageResponse = requests.get(conformanceLink['href'])
conformance = json.loads(conformancePageResponse.text)
print(json.dumps(conformance['conformsTo'], indent=4))

[
    "http://www.opengis.net/spec/ogcapi-common/1.0/conf/core",
    "http://www.opengis.net/spec/ogcapi-common/1.0/conf/json",
    "http://www.opengis.net/spec/ogcapi-common/1.0/conf/html",
    "http://www.opengis.net/spec/ogcapi-common/1.0/conf/oas3",
    "http://www.opengis.net/spec/ogcapi-common-2/1.0/conf/collections",
    "http://www.opengis.net/spec/ogcapi-common-2/1.0/conf/geojson",
    "http://www.opengis.net/spec/ogcapi-common-2/1.0/conf/html",
    "http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core",
    "http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30",
    "http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/html",
    "http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson",
    "http://www.opengis.net/spec/ogcapi_coverages-1/1.0/conf/core",
    "http://www.opengis.net/spec/ogcapi-coverages-1/1.0/conf/html",
    "http://www.opengis.net/spec/ogcapi-coverages-1/1.0/conf/oas30",
    "http://www.opengis.net/t15/opf-styles-1/1.0/conf/core",
   

Only some of that is applicable to Coverages:

In [6]:

for link in conformance['conformsTo']:
    if link.startswith('http://www.opengis.net/spec/ogcapi-common/1.0/'):
        print(link)
    if link.startswith('http://www.opengis.net/spec/ogcapi-coverages-1/1.0/') or link.startswith('http://www.opengis.net/spec/ogcapi_coverages-1/1.0/'):
        print(link)



http://www.opengis.net/spec/ogcapi-common/1.0/conf/core
http://www.opengis.net/spec/ogcapi-common/1.0/conf/json
http://www.opengis.net/spec/ogcapi-common/1.0/conf/html
http://www.opengis.net/spec/ogcapi-common/1.0/conf/oas3
http://www.opengis.net/spec/ogcapi_coverages-1/1.0/conf/core
http://www.opengis.net/spec/ogcapi-coverages-1/1.0/conf/html
http://www.opengis.net/spec/ogcapi-coverages-1/1.0/conf/oas30


Its not very consistent in the spec (PDF, Publication Date: 2020-08-03) whether its underscores or dashes. 


Collections
-----------

Collections is the start of the data. Let's take a look.


In [7]:
collectionsLink = next(link for link in landingPage['links'] if (link['rel'] == 'data' and link['type'] == 'application/json'))
print(json.dumps(collectionsLink, indent=4))

{
    "href": "https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa/collections?f=json",
    "rel": "data",
    "type": "application/json",
    "title": "the set of available collections"
}


The `href` is always `/collections`, but its nice to use the link.

In [8]:
collectionsPageResponse = requests.get(collectionsLink['href'])
collections = json.loads(collectionsPageResponse.text)

The collections list can contain a lot of different entries, only some of which are at least vaguely like a coverage. There is no required link relation, but `coverage` is suggested / recommended.

In [9]:
coverageResults = {}
for collection in collections['collections']:
    linksForCoverage = []
    for link in collection['links']:
        if (link['rel'] == 'coverage') or (link['type'] == 'image/geo+tiff') or (link['type'] == 'image/tiff; application=geotiff'):
            # print(collection['id'] + ": " + collection['title'])
            # print(json.dumps(link, indent=4))
            linksForCoverage.append(link['href'])
    if len(linksForCoverage) > 0:
        coverageResults[collection['id']] = linksForCoverage
print(coverageResults)

{'Daraa_DTED': ['https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa/collections/Daraa_DTED/coverage?f=geotiff'], 'Daraa_mosaic_2019': ['https://test.cubewerx.com/cubewerx/cubeserv/demo/ogcapi/Daraa/collections/Daraa_mosaic_2019/coverage?f=geotiff']}


In [12]:
from PIL import Image
from PIL import ImageStat
import io

for coverage in coverageResults.keys():
    print(coverage)
    for url in coverageResults[coverage]:
        requestResponse = requests.get(url)
        geotiffData = requestResponse.content
        im = Image.open(io.BytesIO(geotiffData))
        print(im.format, im.size, im.mode)
        imagestats = ImageStat.Stat(im)
        print(imagestats.extrema, imagestats.mean, imagestats.median, imagestats.stddev)
        im.show()


Daraa_DTED
TIFF (2401, 1201) I
[(0, 255)] [110.08577816417736] [113] [47.10232367071073]
Daraa_mosaic_2019


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fa7e8112bf8>